In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors, KDTree
from scores import scores
import time
import warnings
warnings.filterwarnings("ignore")

### 4.) Import Data

In [2]:
%run ../notebooks/utils.ipynb

In [3]:
trainandknn_Xy_original_df = pd.read_csv("../data/train_new.csv", sep="|")
train_Xy_original_df, knn_Xy_original_df = train_test_split(trainandknn_Xy_original_df,train_size=0.75, random_state=42) # if FINAL_SUBMISSION else 0.8) #small
test_X_original_df  = pd.read_csv("../data/test.csv", sep="|").iloc[0:500] #TODO: For faster testing we use less data from the test set
test_final_X_df = pd.read_csv("../data/test.csv", sep="|")

train_Xy_wo_knn_df = pd.read_csv("../data/train_new.csv", sep="|")

#Only for test routines
val_Xy_original_df = pd.read_csv("../data/val_new.csv", sep="|")
train_complete_Xy_original_df = pd.read_csv("../data/train.csv", sep="|")

In [4]:
#convention for variables names: datasetname_columntype_transformstatus_dataframeornot
train_y_original_df = train_Xy_original_df[["fraud"]].copy()
train_X_original_df = train_Xy_original_df.copy().drop("fraud", axis=1)

knn_y_original_df = knn_Xy_original_df[["fraud"]].copy()
knn_X_original_df = knn_Xy_original_df.copy().drop("fraud", axis=1)

# Only for test routie#nes
val_y_originial_df = val_Xy_original_df[["fraud"]].copy()
val_X_originial_df = val_Xy_original_df.copy().drop("fraud", axis=1)

train_y_wo_knn_df = train_Xy_wo_knn_df[["fraud"]].copy()
train_X_wo_knn_df = train_Xy_wo_knn_df.copy().drop("fraud", axis=1)

train_complete_y_originial_df = train_complete_Xy_original_df[["fraud"]].copy()
train_complete_X_originial_df = train_complete_Xy_original_df.copy().drop("fraud", axis=1)

In [5]:
#scaler = MinMaxScaler()
scaler = StandardScaler()
transformer = DataTransformer(scaler)

# Adding new Features to train and test set
train_X_unscaled_df = transformer.add_features(train_X_original_df)
test_X_unscaled_df = transformer.add_features(test_X_original_df)
knn_X_unscaled_df = transformer.add_features(knn_X_original_df)

val_X_unscaled_df = transformer.add_features(val_X_originial_df)
train_complete_X_unscaled_df = transformer.add_features(train_complete_X_originial_df) 

transformer.fit_scaler(transformer.add_features(train_complete_X_unscaled_df.append(test_X_unscaled_df, sort=False)))
train_X_scaled_df = transformer.apply_scaler(train_X_unscaled_df)
knn_X_scaled_df   = transformer.apply_scaler(knn_X_unscaled_df)


test_X_scaled_df  = transformer.apply_scaler(test_X_unscaled_df)
val_X_scaled_df = transformer.apply_scaler(val_X_unscaled_df)
train_complete_X_scaled_df = transformer.apply_scaler(train_complete_X_unscaled_df)

train_X_wo_knn_unscaled_df = transformer.add_features(train_X_wo_knn_df.copy())
train_X_wo_knn_scaled_df = transformer.apply_scaler(train_X_wo_knn_unscaled_df)

# labels
train_y_df = train_y_original_df.copy()
val_y_df = val_y_originial_df.copy()
knn_y_df = knn_y_original_df.copy()

train_complete_X_scaled_df = transformer.apply_scaler(transformer.add_features(train_complete_Xy_original_df.copy().drop(columns=['fraud'])))
train_complete_y_df = train_complete_Xy_original_df.copy().fraud

test_final_X_df = transformer.add_features(test_final_X_df)


display(train_complete_X_scaled_df.head(5))
display(train_complete_y_df.head(5))

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,totalScannedLineItems
0,0.925130,0.240558,0.153259,0.449502,-1.558114,0.300299,-0.117812,-0.128310,-0.371627,1.559426
1,-0.249301,-1.548098,-0.789781,-0.126199,-0.923450,0.886361,0.244945,0.043534,-0.282261,-0.173782
2,-0.249301,1.114087,0.410578,-0.701901,1.615204,1.472424,-0.185091,-0.137604,-0.379817,-0.289330
3,1.512346,1.634045,1.450544,0.737353,-0.288787,0.886361,-0.158033,-0.128614,-0.345007,1.559426
4,0.925130,-0.939274,1.078709,-0.701901,0.663209,-0.285764,0.007505,-0.010833,-0.472189,1.328332


0    0
1    0
2    0
3    0
4    0
Name: fraud, dtype: int64

## Scoring functions
Defining multiple scores which should be tracked in the HyperParamSearch Object

In [6]:
score = scores.Scores()

In [7]:
scoring = {'AUC': 'roc_auc', 'FBeta': metrics.make_scorer(metrics.fbeta_score, beta=0.5172), "Precision":'precision', "Recall": 'recall', "AP": score.average_precision, "DMC" : score.dmc_score}

## Defining the paramteres which should be tuned
To tune the hyperparameters, i looked into the [documentation here](https://xgboost.readthedocs.io/en/latest/parameter.html#parameters-for-tree-booster). `randint` can be used for integer values, for float values, use `uniform`. 

You can also use a Grid search on single parameters to get a feeling for a good interval. If you want to try only two possibilities, you can create a list like for the `scale_pos_weight` parameter.

**Note: For the classifiers which work without gpu support, you can probably set a parameter n_jobs=-1 to use all processors**

In [8]:
params = {
    "max_depth": randint(2, 6),  # default 3
    "n_estimators": randint(200, 450),  # default 100
    "learnin_rate" :  uniform(0.1, 0.7),
    "gamma" : uniform(0,2),
    "min_child_weight" : uniform(0,50),
    "max_delta_step" : uniform(0,10), #Set it to value of 1-10 might help control the update.
    "reg_lambda" : uniform(0.1,2),
}

## Creating a classifier with some default values
Not all paramters of a classifier should be fine tuned. For SVM for example, the `kernel`-paramter should be set manually. In the case of xgboost, some things like the objective, the booster and the tree method should not be tuned. The choice of paramters depend on the specific classifier

In [11]:
from xgboost import XGBClassifier
default_xgb = XGBClassifier(booster="gbtree",tree_method='gpu_hist', disable_default_eval_metric=1,objective='binary:logistic',eval_metric='aucpr', n_gpus=1, verbosity=2)
search = RandomizedSearchCV(default_xgb, scoring=scoring, param_distributions=params, random_state=42, n_iter=500,
                            cv=3, verbose=1, n_jobs=-1, return_train_score=True,refit='DMC')

In [13]:
search = RandomizedSearchCV(default_xgb, scoring=scoring, param_distributions=params, random_state=42, n_iter=5000,
                            cv=3, verbose=1, n_jobs=-1, return_train_score=True,refit='DMC')
search.fit(train_complete_X_scaled_df, train_complete_y_df)
results = search.cv_results_

Fitting 3 folds for each of 5000 candidates, totalling 15000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   22.6s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:   51.5s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 1776 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 2426 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 3176 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 4026 tasks      | elapsed:  7.4min
[Parallel(n_jobs=-1)]: Done 4976 tasks      | elapsed:  9.3min
[Parallel(n_jobs=-1)]: Done 6026 tasks      | elapsed: 11.2min
[Parallel(n_jobs=-1)]: Done 7176 tasks      | elapsed: 13.3min
[Parallel(n_jobs=-1)]: Done 8426 tasks      | elapsed: 15.6min
[Parallel(n_jobs=-1)]: Done 9776 tasks      | elapsed: 18.1min
[Parallel(n_jobs=-1)]: Done 11226 tasks      

In [14]:
train_complete_Xy_scaled_df = train_complete_X_scaled_df.copy()
train_complete_Xy_scaled_df['fraud'] =train_complete_y_df.copy()

train_Xy_wo_knn_scaled_df = train_X_wo_knn_scaled_df.copy()
train_Xy_wo_knn_scaled_df['fraud'] = train_y_wo_knn_df

val_Xy_scaled = val_X_scaled_df.copy()
val_Xy_scaled['fraud'] = val_y_df.copy()

In [15]:
scorings = {'AUC': 'roc_auc', 'FBeta': metrics.make_scorer(metrics.fbeta_score, beta=0.5172), "Precision":'precision', "Recall": 'recall', "AP": score.average_precision, "DMC" : score.dmc_score}
xgbo = search.best_estimator_
result_dict = test_classification(xgbo,df_train=train_Xy_wo_knn_scaled_df, df_val=val_Xy_scaled)

Results Fix Split: 
DMC Score: -20  ---  Normalized DMC Score: -0.05319148936170213, 

Results Cross Validation: 
DMC Score: 42.0  ---  Normalized DMC Score: 0.11165248226950353 


In [16]:
dmc = np.mean(search.cv_results_['mean_test_DMC'])
ap = np.mean(search.cv_results_['mean_test_AP'])
precision = np.mean(search.cv_results_['mean_test_Precision'])
recall = np.mean(search.cv_results_['mean_test_Recall'])

In [17]:
print("DMC: {} --- Average Precision: {} --- Precision: {} --- Recall: {}".format(dmc, ap, precision, recall))

DMC: -87.93675199574243 --- Average Precision: 0.844044914297449 --- Precision: 0.5144370515276927 --- Recall: 0.3709619890787609


In [18]:
search.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, disable_default_eval_metric=1,
       eval_metric='aucpr', gamma=0.8759954957911082,
       learnin_rate=0.31522352112346824, learning_rate=0.1,
       max_delta_step=9.849378689708635, max_depth=2,
       min_child_weight=1.9153733419830676, missing=None, n_estimators=220,
       n_gpus=1, n_jobs=1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=0.11118831176373903,
       scale_pos_weight=1, seed=None, silent=True, subsample=1,
       tree_method='gpu_hist', verbosity=2)

In [19]:
search.best_params_

{'gamma': 0.8759954957911082,
 'learnin_rate': 0.31522352112346824,
 'max_delta_step': 9.849378689708635,
 'max_depth': 2,
 'min_child_weight': 1.9153733419830676,
 'n_estimators': 220,
 'reg_lambda': 0.11118831176373903}